#### Prompt Destacado -> Como começar o monitoramento das suas LLM's através dos seus prompts utilizando watsonx.AI e OpenScale

##### Neste guia, utilizaremos um prompt de summarização focado em resumir chamados de um call center. Tanto o prompt quanto a base de dados foram gerados através de IA generativa.

###### A ideia principal aqui é demonstrar a possibilidade de subir um prompt, de qualquer provedor de LLM, no watsonx.ai para futuramente começarmos a monitorar e gerar métricas para avaliarmos o desempenho do prompt/modelo.

In [2]:
### Instalando biblioteca necessária
!pip install --upgrade ibm-aigov-facts-client

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.2
    Uninstalling requests-2.32.2:
      Successfully uninstalled requests-2.32.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibm-cos-sdk-core 2.13.6 requires requests<2.32.3,>=2.32.0, but you have requests 2.32.5 which is incompatible.
ibm-watson-machine-learning 1.0.367 requires pandas<2.2.0,>=0.24.2, but you have pandas 2.2.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#### Dotenv para lermor nossas chaves
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
#### Chaves necessárias
API_KEY = os.getenv('API_KEY') ### Chave de API do IAM da IBM Cloud
SPACE_ID = os.getenv('SPACE_ID') ### ID referente ao espaço de implementação criado dentro do watsonx ai
PROJECT_ID = os.getenv('PROJECT_ID') ### Esse ID é referente ao ID do projeto criado dentro do watsonx ai
OPENSCALE_ID = os.getenv('OPENSCALE_ID') ### Esse ID é encontrado nas definições da sua instancia do Watson Openscale na IBM Cloud
ML_URL = "https://us-south.ml.cloud.ibm.com"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
OPENSCALE_API_URL = "https://api.aiopenscale.cloud.ibm.com"

In [4]:
#### Aqui faremos a conexão com o watsonx.governance através da nossa APIKEY e PROJECT id

from ibm_aigov_facts_client import AIGovFactsClient

facts_client = AIGovFactsClient(
    #region="dallas", ### Caso sua conta da ibm cloud estiver em outro região -> descomentar essa linha e alterar a região
    api_key=API_KEY,
    container_id=PROJECT_ID,
    container_type="project",
    disable_tracing=True
)

In [5]:
### Anexamos então o nosso prompt em si juntamente com algum dos seus metadados, como o provedor, o nome do modelo e etc.
### Vale ressaltar aqui tambem a importancia de especificarmos o tipo de prompt que temos aqui - no nosso exemplo se trata de summarization - 

from ibm_aigov_facts_client import DetachedPromptTemplate, PromptTemplate

detached_information = DetachedPromptTemplate(
    prompt_id="detached_prompt",
    model_id="openai/gpt-4o-2024-08-06",
    model_provider="OpenAI",
    model_name="gpt-4o",
    model_url="https://platform.openai.com/docs/models/gpt-4o",
    prompt_url="prompt_url",
    prompt_additional_info={"model_owner": "João Jardim"}
)

task_id = "summarization" ### Aqui você muda para o seu caso de uso, por exemplo summarization e etc
name = "Sumarização de transcrições de call center"
description = "Prompt para sumarizar conversa entre atendente e cliente."
model_id = "openai/gpt-4o-2024-08-06"

# define parameters for PromptTemplate
prompt_variables = {"transcription": ""}
input_prefix= """
[START_SYSTEM_PROMPT] Você é um assistente especialista em análise e resumo de atendimentos de call center. Sua tarefa é ler a transcrição abaixo e gerar uma sumarização clara, objetiva e profissional da interação.

### Diretrizes da sumarização:
1. A sumarização deve conter os seguintes pontos:
   -  Motivo do contato do cliente.
   -  Ações executadas pelo atendente.
   -  Solução final (se houve).
   -  Se aplicável, status pendente ou necessidade de retorno.

2. A sumarização deve ser escrita em terceira pessoa, formal e objetiva.

3. Utilize uma linguagem clara e sem repetir trechos da conversa literal.

---
.
"""
input_prompt = """
Aqui vai o seu prompt
[END_SYSTEM_PROMPT] 
[START_USER_PROMPT] ### Transcrição:{transcription}[END_USER_PROMPT]
"""
output_prefix= """
### Sumarização:
"""

### Depois de declararmos tudo, utilzaremos essa classe importada anteriormente
prompt_template = PromptTemplate(
    input=input_prompt,
    prompt_variables=prompt_variables,
    input_prefix=input_prefix,
    output_prefix=output_prefix
)

In [6]:
# Agora que temos nossa classe podemos de fato enviar nosso prompt para o watsonx.ai através da seguinte função:

pta_details = facts_client.assets.create_detached_prompt(
    model_id=model_id,
    task_id=task_id,
    name=name,
    description=description,
    prompt_details=prompt_template,
    detached_information=detached_information
)
project_pta_id = pta_details.to_dict()["asset_id"]

2025/09/03 09:12:01 INFO : ------------------------------ Detached Prompt Creation Started ------------------------------
2025/09/03 09:12:05 INFO : The detached prompt with ID 6f0f52eb-ad94-4d75-85ff-c472a1e53e9c was created successfully in container_id 2024c1b2-db60-4363-801c-b6032b026432.
